In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import responses
import urllib.request
import queue

In [44]:
hz = 1.1

In [45]:
def time_index(n):
    index = int(round(n*hz))
    value = (n*hz)/index
    return index,value

0.30000000000000004

In [46]:
a = np.array(range(10))

In [47]:
time_index(5)

(6, 0.9166666666666666)

In [33]:
a

array([0, 1, 2, 3, 4, 5])

In [34]:
(a[-1]-a[0])/time_index(5)*5

4.166666666666667

In [ ]:
*n/(index*n)

In [6]:
vehicle_regulation = pd.read_csv('./vehicle_regulation.csv',index_col = 0)

In [7]:
DTGdata = pd.read_csv('./주행 데이터/편현구/편현구.csv',dtype = 'float')

In [8]:
DTGdata.rename(columns={'Unnamed: 0':'time'},inplace=True)

In [35]:
def chagne_heading(heading):
    heading_buffer = abs(heading)*360
    if heading_buffer > 240:
        heading_buffer = 360 - heading_buffer
    return heading_buffer

In [59]:
class long_term_realtime:
    
    def __init__(self,vehicle,hz=1.1):
        self.DTGdata = []
        self.vehicle = vehicle
        self.hz = hz
        
    def get_DTGdata(self,new_DTG):
        self.DTGdata.append(new_DTG)
        
    def time_index(self,n):
        index = int(round(n*self.hz))
        value = (n*self.hz)/index
        return index,value
        
    def chagne_heading(self,heading):
        heading_buffer = abs(heading)*360
        if heading_buffer > 180:
            heading_buffer = 360 - heading_buffer
        return heading_buffer
        
    def check_over_speed(self,DTGdata,time):
        if (DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['accel_lim'] + DTGdata['road_limit'][time]):
            print('과속 중입니다. 속도를 줄이세요.')
            
    def check_long_accel(self,DTGdata,time):
        if (DTGdata[(DTGdata.index >= time-self.time_index(vehicle_regulation[self.vehicle]['long_accel_sec'])[0]) & 
            (DTGdata['velocity'] <= vehicle_regulation[self.vehicle]['accel_lim'] + DTGdata['road_limit'][time])].empty):
            print('장기과속 중입니다. 속도를 줄이세요.')
                        
    def check_rapid_accel(self,DTGdata,time):
        if (((DTGdata['velocity'][time]-DTGdata['velocity'][time-self.time_index(1)[0]])*self.time_index(1)[1] >= vehicle_regulation[self.vehicle]['rapid_accel_acceleration']) &
            (DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_accel_threshold'])):
            print('급 가속 하였습니다.')
            
    def check_rapid_start(self,DTGdata,time):
        if (((DTGdata['velocity'][time]-DTGdata['velocity'][time-self.time_index(1)[0]])*self.time_index(1)[1] >= vehicle_regulation[self.vehicle]['rapid_start_acceleration']) &
            (DTGdata['velocity'][time] <= vehicle_regulation[self.vehicle]['rapid_start_threshold'])):
            print('급 출발 하였습니다.')
        
    def check_rapid_decel(self,DTGdata,time):
        if (((DTGdata['velocity'][time-self.time_index(1)[0]]-DTGdata['velocity'][time])*self.time_index(1)[1] >= vehicle_regulation[self.vehicle]['rapid_decel_deceleration']) &
            (DTGdata[ 'velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_decel_threshold'])):
            print('급 감속 하였습니다.')

    def check_rapid_stop(self,DTGdata,time):
        if (((DTGdata['velocity'][time]-DTGdata['velocity'][time-self.time_index(1)[0]]) >= vehicle_regulation[self.vehicle]['rapid_start_acceleration']) &
            (DTGdata['velocity'][time] <= vehicle_regulation[self.vehicle]['rapid_start_threshold'])):
            print('급 정지 하였습니다.')
        
    def check_rapid_lanechange(self,DTGdata,time):
        if ((DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_lanechange_threshold']) & 
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(1)[0]])*self.time_index(1)[1] >= vehicle_regulation[self.vehicle]['rapid_lanechange_angle']) & 
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(5)[0]])*self.time_index(5)[1]/vehicle_regulation[self.vehicle]['rapid_lanechange_time'] <= vehicle_regulation[self.vehicle]['rapid_lanechange_cangle']) &
            (abs(DTGdata['velocity'][time] - DTGdata['velocity'][time-self.time_index(1)[0]])*self.time_index(1)[1] <= vehicle_regulation[self.vehicle]['rapid_lanechange_defference'])):
            print('급 차선변경 하였습니다.')
                               
    def check_rapid_overtake(self,DTGdata,time):
        if ((DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_overtake_threshold']) & 
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(1)[0]])*self.time_index(1)[1] >= vehicle_regulation[self.vehicle]['rapid_overtake_angle']) & 
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(5)[0]])*self.time_index(5)[1]/vehicle_regulation[self.vehicle]['rapid_lanechange_time'] <= vehicle_regulation[self.vehicle]['rapid_overtake_cangle']) &
            (abs(DTGdata['velocity'][time] - DTGdata['velocity'][time-self.time_index(1)[0]])*self.time_index(1)[1]>= vehicle_regulation[self.vehicle]['rapid_overtake_acceleration'])):
            print('급 앞지르기 하였습니다.')
        
    def check_rapid_turn(self,DTGdata,time):
        if ((DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_turn_threshold']) &
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(vehicle_regulation[self.vehicle]['rapid_turn_sec'])[0]])*self.time_index(vehicle_regulation[self.vehicle]['rapid_turn_sec'])[1] >= vehicle_regulation[self.vehicle]['rapid_turn_angle_start'])&
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(vehicle_regulation[self.vehicle]['rapid_turn_sec'])[0]])*self.time_index(vehicle_regulation[self.vehicle]['rapid_turn_sec'])[1] <= vehicle_regulation[self.vehicle]['rapid_turn_angle_end'])):
            print('급 회전 하였습니다.')
        
    def check_rapid_uturn(self,DTGdata,time):
        if ((DTGdata['velocity'][time] >= vehicle_regulation[self.vehicle]['rapid_uturn_threshold']) &
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(vehicle_regulation[self.vehicle]['rapid_uturn_sec'])[0]])*self.time_index(vehicle_regulation[self.vehicle]['rapid_uturn_sec'])[1] >= vehicle_regulation[self.vehicle]['rapid_uturn_angle_start'])&
            (self.chagne_heading(DTGdata['heading'][time] - DTGdata['heading'][time-self.time_index(vehicle_regulation[self.vehicle]['rapid_uturn_sec'])[0]])*self.time_index(vehicle_regulation[self.vehicle]['rapid_uturn_sec'])[1] <= vehicle_regulation[self.vehicle]['rapid_uturn_angle_end'])):
            print('급 U턴 하였습니다.')
        
#     def check_long_term(self,DTGdata,time):
        
        
    def check_DTGdata(self,new_DTG):
        self.get_DTGdata(new_DTG)
        DTGdata = pd.DataFrame(self.DTGdata,columns = ['time', 'velocity', 'rpm', 'brake', 'gpsX', 'gpsY', 'heading', 'accelX',
       'accelY', 'road_limit'])
        length = len(DTGdata)
        time = new_DTG[0]
        
        if ((time >= self.time_index(vehicle_regulation[self.vehicle]['rapid_uturn_sec'])[0])&
             (time <= self.time_index(vehicle_regulation[self.vehicle]['long_accel_sec'])[0])):
            self.check_over_speed(DTGdata,time)
            self.check_rapid_accel(DTGdata,time)
            self.check_rapid_start(DTGdata,time)
            self.check_rapid_decel(DTGdata,time)
            self.check_rapid_stop(DTGdata,time)
            self.check_rapid_lanechange(DTGdata,time)
            self.check_rapid_overtake(DTGdata,time)
            self.check_rapid_turn(DTGdata,time)
            self.check_rapid_uturn(DTGdata,time)

        elif((time >= self.time_index(vehicle_regulation[self.vehicle]['long_accel_sec'])[0])&
            (time <= self.time_index(vehicle_regulation[self.vehicle]['long_term_time'])[0])):
            self.check_over_speed(DTGdata,time)
            self.check_long_accel(DTGdata,time)
            self.check_rapid_accel(DTGdata,time)
            self.check_rapid_start(DTGdata,time)
            self.check_rapid_decel(DTGdata,time)
            self.check_rapid_stop(DTGdata,time)
            self.check_rapid_lanechange(DTGdata,time)
            self.check_rapid_overtake(DTGdata,time)
            self.check_rapid_turn(DTGdata,time)
            self.check_rapid_uturn(DTGdata,time)
            
        if(time >= self.time_index(vehicle_regulation[self.vehicle]['long_term_time'])[0]):
            self.check_over_speed(DTGdata,time)
            self.check_long_accel(DTGdata,time)
            self.check_rapid_accel(DTGdata,time)
            self.check_rapid_start(DTGdata,time)
            self.check_rapid_decel(DTGdata,time)
            self.check_rapid_stop(DTGdata,time)
            self.check_rapid_lanechange(DTGdata,time)
            self.check_rapid_overtake(DTGdata,time)
            self.check_rapid_turn(DTGdata,time)
            self.check_rapid_uturn(DTGdata,time)
            check_long_term(DTGdata,time)

In [64]:
taxi_realtime = long_term_realtime('taxi',hz=1.4)

In [65]:
for i in range(10000):
    taxi_realtime.check_DTGdata(DTGdata.iloc[i].values)

급 출발 하였습니다.
급 정지 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 차선변경 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 회전 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 감속 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
급 차선변경 하였습니다.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이세요.
과속 중입니다. 속도를 줄이

IndexError: single positional indexer is out-of-bounds